In [ ]:
import dotenv
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
# assert version is > 1.0.0
assert openai.__version__ > "1.0.0", "Please upgrade the OpenAI Python client: pip install -U openai"
print(openai.__version__)

1.1.1


## Files for context of the assistant

In [9]:
import glob
# get all python files inside data/tinygrad even inside subfolders etc
filepath = "data/tinygrad/"
python_files = glob.glob(filepath + "**/*.py", recursive=True)
print(f"Found {len(python_files)} python files")

def is_usefull_python_file(python_file_path):
    # just to remove useless file for the assistant 
    bad_strings_to_not_match = ["test", "mlperf", "setup.py",
                                "runtime", "examples", "renderer",
                                 "openpilot", "models","adreno", 
                                 "extra", "jit", "sz.py"]
    if "resnet.py" in python_file_path:
        return True
    for bad_string in bad_strings_to_not_match:
        if bad_string in python_file_path:
            return False
    return True
# print files with model in the path

print("Files with model in the path")
print(list(filter(lambda x: "model" in x, python_files)))


python_files = list(filter(is_usefull_python_file, python_files))
print(f"Found {len(python_files)} python files after filtering")
assert len(python_files) <= 20, "Openai limit is 20 files"
print("\n".join(python_files))

Found 228 python files
Files with model in the path
['data/tinygrad/test/models/test_real_world.py', 'data/tinygrad/test/models/test_train.py', 'data/tinygrad/test/models/test_end2end.py', 'data/tinygrad/test/models/test_onnx.py', 'data/tinygrad/test/models/test_mnist.py', 'data/tinygrad/test/models/test_efficientnet.py', 'data/tinygrad/test/models/test_whisper.py', 'data/tinygrad/test/models/test_waifu2x.py', 'data/tinygrad/test/models/test_rnnt.py', 'data/tinygrad/test/models/test_bert.py', 'data/tinygrad/test/extra/test_export_model.py', 'data/tinygrad/test/external/external_model_benchmark.py', 'data/tinygrad/test/external/external_test_jit_on_models.py', 'data/tinygrad/test/external/external_test_allocator_on_models.py', 'data/tinygrad/models/unet3d.py', 'data/tinygrad/models/efficientnet.py', 'data/tinygrad/models/vit.py', 'data/tinygrad/models/resnet.py', 'data/tinygrad/models/transformer.py', 'data/tinygrad/models/mask_rcnn.py', 'data/tinygrad/models/retinanet.py', 'data/tinygr

## Upload the files to Openai

In [11]:
from openai import OpenAI
from tqdm.auto import tqdm
client = OpenAI()
uploaded_files = []
for filename in tqdm(python_files):
    file_obj = client.files.create(
        file=open(filename, 'rb'),
        purpose='assistants',
        
    )
    uploaded_files.append(file_obj.id)

  0%|          | 0/20 [00:00<?, ?it/s]

In [12]:
uploaded_files

['file-tBjLzvyBuP4dYop5FCSwuEtb',
 'file-Kx0PfJUP7l04XkRZbtYVc16O',
 'file-yVsLs62WAGRKYZZYsw3dOkls',
 'file-eZcVjBZBNkGoaheSrMvqHz1b',
 'file-i7ZYDFJRAVQhmtFyHhBi8BP7',
 'file-VBf84l6dHvWynYZxWVFMYmT6',
 'file-P6HtZVnisbqvIRR866bNCIuM',
 'file-G2bffUxlxY90zBV3VIFCMTCc',
 'file-hTxQ1IhuvXiF1fhQsfD5AXEJ',
 'file-MGpQOsAJXhwZF2jTP75YvfFf',
 'file-TKO1zhJIYtScKL4Ztr0aCO5h',
 'file-opzlSYW2vHONow5h2a7C1Y8b',
 'file-69YpFEWAwZB4lxQOs3EfgFQ0',
 'file-Cp9OPRhTFfcXhoICHXCrZLW3',
 'file-oqm4EHOwpTOjt9dYGSthuHrn',
 'file-WuqoF7SwLYmjKLx5EuAV5JzO',
 'file-5cDOsjWn9R95PZzxvtsvT28L',
 'file-jv1zfst9aK8VYdkg7uTZB6re',
 'file-f7lhFDJRKvvFgPxDmUMk6PuH',
 'file-OSqH9D4JVi2WN59gsfxCI8Zn']

## Create the assistant

In [13]:
tinygrad_assistant = openai.beta.assistants.create(
    instructions="You are helping coders understand the tinygrad library. You have access to the source code of the codebase. You are a helpful tinygrad assistant.",
    name="Tinygrad Assistant (API)",
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    model="gpt-4-1106-preview",
    file_ids=uploaded_files,
)

## Create an empty thread
Empty threads are not linked to an assistant yet. Multiple assistants can be used inside the same thread :)

In [14]:
thread = openai.beta.threads.create()
thread_id = thread.id

In [16]:
query = "Explain Conv2d implementation in tinygrad. Return the corresponding Code for each explanation"

In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread_id,
    role="user",
    content=query,
)
run = client.beta.threads.runs.create(
    thread_id=thread_id,
    assistant_id=tinygrad_assistant.id)

import time
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)
    if run.completed_at:
        elapsed = run.completed_at - run.created_at
        elapsed = time.strftime("%H:%M:%S", time.gmtime(elapsed))
        print(f"Run completed in {elapsed}")
        break
    print("Waiting 1sec")
    time.sleep(1)

Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Waiting 1sec
Run completed in 00:00:16


In [18]:
messages = client.beta.threads.messages.list(thread_id=thread_id)
last_message = messages.data[0]

text = last_message.content[0].text.value
print(text)

The `Conv2d` class in tinygrad is defined with the following key components:

### Initialization (`__init__` method)

```python
class Conv2d:
  def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
    self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else tuple(kernel_size)
    self.stride, self.padding, self.dilation, self.groups = stride, padding, dilation, groups
    self.weight = self.initialize_weight(out_channels, in_channels // groups, *self.kernel_size)
    assert all_int(self.weight.shape), "does not support symbolic shape"
    bound = 1 / math.sqrt(prod(self.weight.shape[1:]))
    self.bias = Tensor.uniform(out_channels, low=-bound, high=bound) if bias else None
```

- The constructor takes several arguments such as the number of input channels (`in_channels`), the number of output channels (`out_channels`), the kernel size (`kernel_size`), stride, padding, dilation, and whether to use gr